In [2]:
!pip install colorlog  
!pip install pytesseract
!pip install playwright
!pip install nltk
!pip install esprima
import sqlite3
from bs4 import BeautifulSoup
import os
print(os.getcwd())
os.chdir('/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm')
print(os.getcwd())
from src.main import main
from src.utils.helpers import truncate_text, clean_string
from src.utils.logger import setup_logger
from src.llm.access_2_cluster import Access2Cluster
from src.data.html_processor import extract_html_info

logger = setup_logger(__name__, level='DEBUG') # Change to 'INFO' for less verbosity

/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/notebooks
/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm


/Users/niklasquendt/anaconda3/envs/bda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
# %%

from src.main import main
from src.utils.helpers import truncate_text, clean_string
from src.utils.logger import setup_logger
from src.llm.access_2_cluster import Access2Cluster
from src.data.html_processor import extract_html_info

logger = setup_logger(__name__, level='DEBUG') # Change to 'INFO' for less verbosity


# %%

import os
import shutil
import sqlite3
import random
# Pfad zur SQLite-Datenbank
db_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/playwright_script.db'

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# SQL-Abfrage zum Abrufen der Spaltennamen
cursor.execute("PRAGMA table_info(tests)")
columns_info = cursor.fetchall()

# Schließen der Verbindung zur Datenbank
conn.close()

# Spaltennamen extrahieren
column_names = [column[1] for column in columns_info]

# Ausgabe der Spaltennamen
print(f"Spaltennamen der Tabelle 'tests': {column_names}")

# Pfad zur SQLite-Datenbank
db_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/playwright_script.db'

# Pfade zu den Verzeichnissen
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

html_dir = 'html'
screenshot_dir = 'screenshots'

# Verzeichnisstruktur erstellen
def create_dir_structure(base, html_subdir, screenshot_subdir):
    if not os.path.exists(base):
        os.makedirs(base)
    if not os.path.exists(os.path.join(base, html_subdir)):
        os.makedirs(os.path.join(base, html_subdir))
    if not os.path.exists(os.path.join(base, screenshot_subdir)):
        os.makedirs(os.path.join(base, screenshot_subdir))

create_dir_structure(train_dir, html_dir, screenshot_dir)
create_dir_structure(test_dir, html_dir, screenshot_dir)

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Daten aus der Datenbank abrufen
cursor.execute("SELECT id, steps, html, screenshot FROM tests") # SQL-Abfrage
rows = cursor.fetchall()

# Schließen der Verbindung zur Datenbank
conn.close()

# Zufällige Durchmischung der Liste
random.shuffle(rows)

# Split bei 50/20 (da insgesamt 70 Tests)
train_tests = rows[:50]
test_tests = rows[50:]

# Hilfsfunktion zum Kopieren von Dateien
def copy_files(tests, target_dir):
    for test in tests:
        test_id, text, html_file, screenshot_file = test
        # HTML-Datei kopieren
        if html_file and os.path.exists(html_file):
            shutil.copy(html_file, os.path.join(target_dir, html_dir, os.path.basename(html_file)))
        # Screenshot-Datei kopieren
        if screenshot_file and os.path.exists(screenshot_file):
            shutil.copy(screenshot_file, os.path.join(target_dir, screenshot_dir, os.path.basename(screenshot_file)))

# Dateien in die entsprechenden Verzeichnisse kopieren
copy_files(train_tests, train_dir)
copy_files(test_tests, test_dir)

print(f"Anzahl der Trainingstests: {len(train_tests)}")
print(f"Anzahl der Testtests: {len(test_tests)}")




Spaltennamen der Tabelle 'tests': ['id', 'steps', 'expectation', 'html', 'screenshot', 'test_script']
Anzahl der Trainingstests: 50
Anzahl der Testtests: 50


In [10]:
# ignore


import sqlite3

# Pfad zur SQLite-Datenbank
db_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/playwright_script.db'

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# SQL-Abfrage zum Zählen der Anzahl der Tests
cursor.execute("SELECT COUNT(*) FROM tests")
num_tests = cursor.fetchone()[0]

# Schließen der Verbindung zur Datenbank
conn.close()

# Ausgabe der Anzahl der Tests
print(f"Anzahl der Tests in der Datenbank: {num_tests}")

Anzahl der Tests in der Datenbank: 100


In [12]:
#ignore 


import sqlite3
import random
import json
import os

# Pfad zur SQLite-Datenbank und zu den Ausgabedateien
db_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/playwright_script.db'
train_output_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/train_tests.json'
test_output_path = '/Users/niklasquendt/Documents/Uni/PSDA/Übung 3/LLM Cadenza neu/cadenza-playwright-llm/data/data/test_tests.json'

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Daten aus der Datenbank abrufen
cursor.execute("SELECT * FROM tests")
rows = cursor.fetchall()

# Schließen der Verbindung zur Datenbank
conn.close()

# Zufällige Durchmischung der Liste
random.shuffle(rows)

# Split bei 60/10
train_tests = rows[:60]
test_tests = rows[60:]

# Umwandlung in eine verständlichere Struktur
train_tests_json = [{"id": row[0], "text": row[1], "image": row[2]} for row in train_tests]
test_tests_json = [{"id": row[0], "text": row[1], "image": row[2]} for row in test_tests]

# Speichern der geteilten Tests in separaten JSON-Dateien
with open(train_output_path, 'w') as f:
    json.dump(train_tests_json, f, indent=4)

with open(test_output_path, 'w') as f:
    json.dump(test_tests_json, f, indent=4)

print(f"Anzahl der Trainingstests: {len(train_tests)}")
print(f"Anzahl der Testtests: {len(test_tests)}")


Anzahl der Trainingstests: 60
Anzahl der Testtests: 40


In [ ]:
#ignore

import sqlite3
import random
import json
import os

# Pfad zur SQLite-Datenbank und zu den Ausgabedateien
db_path = '../data/playwright_script.db'
train_output_path = '../data/train_tests.json'
test_output_path = '../data/test_tests.json'

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Daten aus der Datenbank abrufen
cursor.execute("SELECT * FROM tests")
rows = cursor.fetchall()

# Schließen der Verbindung zur Datenbank
conn.close()

# Zufällige Durchmischung der Liste
random.shuffle(rows)

# Split bei 60/10
train_tests = rows[:60]
test_tests = rows[60:]

# Umwandlung in eine verständlichere Struktur
train_tests_json = [{"id": row[0], "text": row[1], "image": row[2]} for row in train_tests]
test_tests_json = [{"id": row[0], "text": row[1], "image": row[2]} for row in test_tests]

# Speichern der geteilten Tests in separaten JSON-Dateien
with open(train_output_path, 'w') as f:
    json.dump(train_tests_json, f, indent=4)

with open(test_output_path, 'w') as f:
    json.dump(test_tests_json, f, indent=4)

print(f"Anzahl der Trainingstests: {len(train_tests)}")
print(f"Anzahl der Testtests: {len(test_tests)}")
